# Задание 2


Вам необходимо построить модель, которая на основании данных, поступающих каждую минуту, определяют качество продукции, производимое на обжиговой машине.
Обжиговая машина представляет собой агрегат, состоящий из 5 одинаковых по размеру камер, в каждой камере установлено по 3 датчика температур. Кроме этого, для данной задачи Вы собрали данные о высоте слоя сырья и его влажности. Высота слоя и влажность измеряются при входе сырья в машину. Сырье проходит через обжиговую машину за час.
Данные с показателями работы обжиговой машины содержатся в файле X_data.csv:
Качество продукции измеряется в лаборатории по пробам, которые забираются каждый час, данные по известным анализам содержатся в файле  . В файле указано время забора пробы, проба забирается на выходе из обжиговой машины.
Вы договорились с заказчиком, что оценкой модели будет являться показатель MAE, для оценки модели необходимо сгенерировать предсказания за период, указанный в файле Y_submit.csv (5808 предиктов).
Все необходимые данные лежат по ссылке: 
https://www.dropbox.com/s/lo2w549fv8lo3oc/Test2.7z

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [2]:
data_all = pd.read_csv('X_data.csv', sep=';')
data_all = data_all.rename(columns={'Unnamed: 0':'time'}).set_index('time')
data_all.index = pd.to_datetime(data_all.index)
data_all.head()

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
time,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22
2015-01-01 00:01:00,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22
2015-01-01 00:02:00,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22
2015-01-01 00:03:00,213,211,211,344,351,346,477,473,481,355,349,355,242,241,242,167.34,9.22
2015-01-01 00:04:00,213,211,211,343,350,346,478,473,482,358,349,355,243,241,242,163.04,9.22


In [3]:
data_all.shape

(2103841, 17)

Таблица X_data.csv полностью состоит из более 2 млн строк с числовыми признаками

In [4]:
Y_train = pd.read_csv('Y_train.csv', sep=';',header=None)
Y_train = Y_train.rename(columns={0: 'time', 1: 'target'}).set_index('time')
Y_train.index = pd.to_datetime(Y_train.index)
Y_train.head()

,target
time,
2015-01-04 00:05:00,392
2015-01-04 01:05:00,384
2015-01-04 02:05:00,393
2015-01-04 03:05:00,399
2015-01-04 04:05:00,400


In [5]:
Y_train.shape

(29184, 1)

Таблица  Y_train.scv имеет 29 184 целевых числовых переменных, распределенных в соответствии с  определенными минутами.
Поскольку нам известно 29 тыс временных данных с целевыми переменными и более 2 млн данных по времени с признаками, для построения модели необходимо склеить данные по известному времени.

In [6]:
data = Y_train.merge(data_all, how='left', on='time', validate='1:1')
data.head()

,target,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
time,,,,,,,,,,,,,,,,,,
2015-01-04 00:05:00,392,277,326,273,322,335,352,505,501,670,326,379,337,231,236,242,153.77,7.90
2015-01-04 01:05:00,384,277,253,272,320,333,355,500,501,687,337,396,335,234,242,230,158.27,6.96
2015-01-04 02:05:00,393,262,218,260,326,336,330,505,499,443,347,399,332,243,251,240,153.36,7.29
2015-01-04 03:05:00,399,243,238,252,327,329,308,520,498,540,342,387,334,257,258,246,153.21,7.11
2015-01-04 04:05:00,400,236,238,245,323,320,318,522,501,524,343,371,344,264,263,265,195.71,7.97


In [7]:
Y_submit = pd.read_csv('Y_submit.csv', sep=';',header=None)
Y_submit = Y_submit.rename(columns={0: 'time', 1: 'target'}).set_index('time')
Y_submit.index = pd.to_datetime(Y_submit.index)
Y_submit.head()

,target
time,
2018-05-04 00:05:00,420
2018-05-04 01:05:00,420
2018-05-04 02:05:00,420
2018-05-04 03:05:00,420
2018-05-04 04:05:00,420


In [8]:
Y_submit.shape

(5808, 1)

Необходимо предсказать 5808 целевых переменных в соответствии с данными о минутах. 
Для этого их также объединяем с получившейся общей таблицей

In [9]:
data_submit = Y_submit.merge(data_all, how='left', on='time', validate='1:1')
data_submit.head()

,target,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
time,,,,,,,,,,,,,,,,,,
2018-05-04 00:05:00,420,258,261,256,347,337,352,474,434,443,336,321,516,248,247,236,154.93,6.81
2018-05-04 01:05:00,420,255,263,258,351,-162,359,460,441,454,320,319,341,238,255,236,156.52,7.89
2018-05-04 02:05:00,420,259,261,288,363,104,374,482,474,483,343,325,307,226,249,255,153.37,6.16
2018-05-04 03:05:00,420,272,266,279,373,338,386,495,510,503,330,307,323,213,245,250,161.45,7.53
2018-05-04 04:05:00,420,287,290,309,373,352,358,512,514,499,317,220,319,215,265,249,165.22,6.93


In [56]:
x, y = data.drop(columns='target'), data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=31)

In [57]:
def score(rgr, x_train, x_test, y_train, y_test):
    y_pred_test = rgr.predict(x_test)
    y_pred_train = rgr.predict(x_train)
    #средняя абсолютная ошибка
    print(f"MAE: \t{mean_absolute_error(y_train, y_pred_train):.2f}\t{mean_absolute_error(y_test, y_pred_test):.2f}")

Посмотрим несколько моделей и выберем наиболее подходящую.1 способ - модель линейной регрессии

In [58]:
rgr = LinearRegression()
rgr.fit(x_train, y_train)
score(rgr, x_train, x_test, y_train, y_test)

MAE: 	15.55	15.75


Ошибка довольно большая.  Посмотрим, как с задачей справится модель леса деревьев решений

In [76]:
rgr = RandomForestRegressor(n_estimators=1000)
rgr.fit(x_train, y_train)
score(rgr, x_train, x_test, y_train, y_test)

MAE: 	3.62	9.92


Проверим, насколько лучше справится модель бустинга 

In [77]:
rgr = CatBoostRegressor(n_estimators=1000, learning_rate=0.2, max_depth=4, silent=True)
rgr.fit(x_train, y_train)
score(rgr, x_train, x_test, y_train, y_test)

MAE: 	7.84	9.75


Наименьшая средняя сумма абсолютной разницы между фактическим значением и прогнозируемым значением
получилась в Catboost, результаты модели бустинга и записываем в показания

In [78]:
rgr = CatBoostRegressor(n_estimators=1000, learning_rate=0.2, max_depth=4, silent=True)
rgr.fit(x, y)
to_submit = pd.Series(rgr.predict(x).astype(int), index=x.index).to_frame()
to_submit.to_csv('submit.csv', header=False, sep=';')